In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import shutil

In [5]:
from ultralytics import YOLO
model = YOLO("/content/sample_data/last.pt")

In [ ]:
from paddleocr import PaddleOCR
import cv2
import re
# test_folder = '/kaggle/working/results'
test_folder = "./results"
# !rm -rf ./results
# os.mkdir(test_folder)
CONFIDENCE_THRESHOLD = 0.5

# https://paddlepaddle.github.io/PaddleOCR/latest/en/quick_start.html#use-by-code
ocr = PaddleOCR(use_angle_cls = True, use_gpu = True)

def paddle_ocr(image, x1, y1, x2, y2):
    image = image[max(0, y1-5) : (y2+5), max(0, x1-5) : (x2+5)]
    # image = cv2.resize(image, (int(image.shape[1] * 1.1), int(image.shape[0] * 1.1)), interpolation=cv2.INTER_LINEAR)
    # gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    # sharp = cv2.Laplacian(blurred, cv2.CV_64F)

    # thresh = cv2.adaptiveThreshold(sharp, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    result = ocr.ocr(image, det=True, rec = True, cls = False)
    if result is None  or result[0] is None:
        return
    text_score = []
    print(result[0])
    for line in result[0]:
        l_text, scores = line[1][0], line[1][1]
        if np.isnan(scores):
            scores = 0
        else:  # threshold
            scores = int(scores * 100)
        if 50 < scores:
            text_score.append((l_text, scores))
    text = "".join(t[0] for t in text_score)
    score = np.average([int(t[1]) for t in text_score]) if text_score else 0
    pattern = re.compile('[\W]')
    text = pattern.sub('', text)
    text = text.replace("???", "")
    text = text.replace("O", "0")
    text = text.replace("粤", "")

    # car plate: 7-8 characters
    if len(text) < 9:
        return text, score

def predict_ocr(image):
    """
        image: vehicle image was cropped
    """
    results = model.predict(image, device='cuda')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # plt.imshow(image)
    if results is None:
        return
    # process for single vehicle
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])    # coordinates of plate
            return paddle_ocr(image, x1, y1, x2, y2)

    # process for multiple vehicle
    # for result in results:
    #     filtered_boxes = [box for box in result.boxes if box.conf[0] > CONFIDENCE_THRESHOLD]
    #     for box in filtered_boxes:
    #         x1, y1, x2, y2 = map(int, box.xyxy[0])    # plate
    #         # confidence = box.conf[0]  # Get confidence

    #         # Crop bounding box
    #         # rect = image[y1:y2, x1:x2]
    #         # plt.imshow(rect)
    #         # print((x1, y1), (x2, y2))
    #         # text = paddle_ocr(image, x1, y1, x2, y2)
    #         return paddle_ocr(image, x1, y1, x2, y2)


    #         # print("NONE") if result==None else print(result)
    #         # text = result[0][0]

    #         # print(f"Detected text: {confidence:.2f}_{text}")
    #         # cv2.putText(image, f'{confidence:.2f}_{text}', (x1-30, y1-10),
    #         #         cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    #         # cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 3)

    #         # boxes = [elements[0] for elements in line for line in result]
    #         # txts = [elements[1][0] for elements in line for line in result]
    #         # scores = [elements[1][1] for elements in line for line in result]
    #         # output = draw_ocr(image, boxes, txts, scores, font_path='path_to_font.ttf')

    #         # # text = pytesseract.image_to_string(rect, config='--psm 6')  # perform OCR
    # return paddle_ocr(image, x1, y1, x2, y2)

def predict_folder(path_test):
    for img in os.listdir(path_test):
        image = cv2.imread(os.path.join(path_test, img))
        image = predict_ocr(image)
        cv2.imwrite(os.path.join(test_folder, img), image)

# predict_folder('/kaggle/input/demo-ttt')

### check plate

In [ ]:
list_plate = []
def check_backup(time, name_vehicle, plate_number, score, str_vuotdendo):
  if len(plate_number) in (6, 7, 8, 9):
    list_plate.append((round(time/1000, 2), name_vehicle, plate_number, int(score), str_vuotdendo))


### vượt đèn đỏ

In [ ]:
def check_red_light(image):
  """
    traffic light was cropped
  """
  image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
  lower_red1 = np.array([0, 120, 70])
  upper_red1 = np.array([10, 255, 255])
  lower_red2 = np.array([170, 120, 70])
  upper_red2 = np.array([180, 255, 255])
  mask = cv2.inRange(image, lower_red1, upper_red1) + cv2.inRange(image, lower_red2, upper_red2)
  red_pixels = cv2.countNonZero(mask)
  return True if red_pixels > 500 else False

def check_cross_red_light(x2, y2):
  if 722 < x2 < 1290 and 420 < y2 < 890:
    return True

In [ ]:
import cv2
# !rm -rf /kaggle/working/vid_results
# os.mkdir("/kaggle/working/vid_results")
video_path = "/content/sample_data/test.mp4"
vidcap = cv2.VideoCapture(video_path)

model2 = YOLO("yolo12n.pt")
CONFIDENCE_THRESHOLD = 0.5

vidcap.set(cv2.CAP_PROP_FPS, 60)
vidcap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
vidcap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

fps = int(vidcap.get(cv2.CAP_PROP_FPS))
width = int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = './processed_vid.mp4'
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

if not vidcap.isOpened():
    print("Error opening video stream")
    exit()

frame_count = 0
while True:
    success, frame = vidcap.read()
    if not success:
        break
    # if frame_count % 2 == 0:
    time_in_ms = vidcap.get(cv2.CAP_PROP_POS_MSEC)
    results = model2.predict(frame, device='cuda', classes=[2, 3, 5, 7])
    for result in results:
        filtered_boxes = [box for box in result.boxes if box.conf[0] > CONFIDENCE_THRESHOLD ]
        for box in filtered_boxes:

            str_vuotdendo = ""
            x1, y1, x2, y2 = map(int, box.xyxy[0])   # (x1, y1), (x2, y2): coordinates of the vehicle
            if check_red_light(frame[1290:1335, 210:250]) and check_cross_red_light(x2, y2):
                str_vuotdendo = "vuot den do"


            confidence = box.conf[0]
            cls = int(box.cls[0])
            list_vehicle = {2: "car", 3: "motorbike", 5: "bus", 7: "truck"}
            name_vehicle = list_vehicle[cls]
            vehicle = frame[max(0, y1-5):y2+5, max(0, x1-5):x2+5]     # crop vehicle
            ocr_result = predict_ocr(vehicle)    # text: number plate
            text, score = ocr_result if ocr_result is not None else ("", 0)
            # print(f"Detected text: {confidence:.2f}_{cls}_{text}")


            if text is not None:
                check_backup(time_in_ms, name_vehicle, text, score, str_vuotdendo)
                cv2.putText(frame, f'{confidence:.2f}_{name_vehicle}_{text}{str_vuotdendo}', (x1-10, y1-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                cv2.putText(frame, f'{confidence:.2f}_{name_vehicle}{str_vuotdendo}', (x1-10, y1-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    # frame = predict_ocr(frame)
    # frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    # cv2.imwrite(f"./results/frame_{frame_count}.jpg", frame)
    # frame = cv2.resize(frame, (960, 640))
    # cv2.imshow("Video", frame)
    out.write(frame)
    frame_count += 1
      # if cv2.waitKey(30) & 0xFF == ord('q'):
      #     break

vidcap.release()
out.release()

# cv2.destroyAllWindows()

In [63]:
list_plate
# vehicles_id

[('car', '30073167', 70),
 ('car', '307310', 68),
 ('car', '3071952', 53),
 ('car', '30473962', 82),
 ('car', '3073961', 66),
 ('car', '073952', 88),
 ('car', '30H73952', 81),
 ('car', '20473952', 69),
 ('car', '20N72952', 74),
 ('car', '30M73957', 73),
 ('car', '30M73952', 78),
 ('car', '30W73952', 82),
 ('car', '30473962', 89),
 ('car', '30G75720', 98),
 ('car', '30H73962', 67),
 ('car', '0G1157', 79),
 ('car', '30H73962', 91),
 ('car', '01157元', 81),
 ('car', '30H73962', 87),
 ('car', '30H73962', 92),
 ('car', 'motorl', 89),
 ('car', '30H73962', 96),
 ('car', '30H73962', 93),
 ('truck', '30H73962', 97),
 ('car', '30H73962', 96),
 ('car', '30611570', 91),
 ('car', '30G11570', 97),
 ('car', '30H73962', 96),
 ('car', '30H73962', 95),
 ('car', '30G11570', 97),
 ('car', '30H73962', 97),
 ('car', '30G11570', 93),
 ('car', '30H73962', 96),
 ('car', '30G11570', 97),
 ('car', '30H73962', 97),
 ('car', '30G11570', 98),
 ('car', '30H73962', 95),
 ('car', '30G11570', 98),
 ('car', '30H73962', 9